<a href="https://colab.research.google.com/github/asma100/device-DR-chatbot/blob/main/chatbot_model_googlecolab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask transformers sentence-transformers langchain faiss-cpu langchain-huggingface PyPDF2

In [29]:
import threading
from flask import Flask, request, jsonify
from transformers import pipeline
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import PyPDF2

# Define Flask app
app = Flask(__name__)

# Load the model and setup FAISS retriever
model = SentenceTransformer("all-mpnet-base-v2")

# Step 1: Initialize embeddings model
embedding_model = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")

# Step 2: Load and process PDF into chunks
pdf_path = "/content/abc.pdf"

def extract_text_from_pdf(pdf_path):
    manual_text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            manual_text += page.extract_text()
    return manual_text

def split_text(text, chunk_size=500, overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap)
    return text_splitter.split_text(text)

def load_and_embed_pdf(pdf_path):
    manual_text = extract_text_from_pdf(pdf_path)
    chunks = split_text(manual_text)

    # Generate embeddings for each chunk
    embeddings = [embedding_model.embed_query(chunk) for chunk in chunks]
    embeddings_array = np.vstack(embeddings)

    return chunks, embeddings_array

chunks, embeddings_array = load_and_embed_pdf(pdf_path)

# Step 3: Combine texts and embeddings for FAISS
text_embeddings = [(chunk, embedding) for chunk, embedding in zip(chunks, embeddings_array)]

# Step 4: Initialize FAISS vector store and convert to a retriever
retriever = FAISS.from_embeddings(
    text_embeddings=text_embeddings,
    embedding=embedding_model
).as_retriever()

# Load the Hugging Face model
hf_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")
qa_model = HuggingFacePipeline(pipeline=hf_pipeline)

# Define the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=qa_model,
    retriever=retriever,
    chain_type="map_reduce"
)

def ask_question(query):
    response = qa_chain.invoke({"query": query})
    return response["result"]

# Define an API endpoint
@app.route('/ask', methods=['POST'])
def ask():
    data = request.get_json()
    query = data.get('query')
    if query:
        answer = ask_question(query)
        return jsonify({'response': answer})
    return jsonify({'response': 'No query provided.'}), 400

# Run Flask app on a separate thread
def run_flask():
    app.run(host='0.0.0.0', port=5000)

threading.Thread(target=run_flask).start()


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


In [31]:
!ngrok config add-authtoken 2nRB8RKc6cfXVdKRv5sPuVLF7hf_2tJxBst8wDbRsDh7s7ZDm


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [34]:
!pip install pyngrok
from pyngrok import ngrok

# Specify protocol and port together
public_url = ngrok.connect(5000, "http")
print("Public URL:", public_url)




Public URL: NgrokTunnel: "https://467c-34-19-11-104.ngrok-free.app" -> "http://localhost:5000"
